In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# -------------------------- LIBRARIES NECESSARY IN THIS PROJECT  -------------------------- #

import pandas as pd
import numpy as np
import seaborn as sns
from datetime import *
import zipfile
import pylab as plt

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.tree import DecisionTreeClassifier

from sklearn import tree
from pydot import graph_from_dot_data
from six import StringIO

from statistics import mean, stdev

from IPython.display import Image

from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import GroupShuffleSplit

from sklearn.metrics import accuracy_score

from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit

# -------------------------- FUNCTIONS -------------------------- #

def getDataSet(data_path):
    return pd.read_csv(data_path)

## Upload dataSets

### Members dataset cleasing

In [ ]:
# -------------------------- PATH OF THE DATASETS USED IN THE PROJECT  -------------------------- #
zf = zipfile.ZipFile('/content/drive/My Drive/dataSets/main.zip')

main_data_set_all_fields = pd.read_csv(zf.open('main.csv'))

## Data manipulation and cleansing

In [ ]:
# Check the dataSet fields
main_data_set_all_fields.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32950 entries, 0 to 32949
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              32950 non-null  int64  
 1   member_key              32950 non-null  object 
 2   updated_at              32950 non-null  object 
 3   first_name              32950 non-null  object 
 4   last_name               32950 non-null  object 
 5   disabilities            8383 non-null   object 
 6   SA1                     3751 non-null   float64
 7   price_zone_code         32950 non-null  object 
 8   plan_key                32950 non-null  object 
 9   plan_status             32950 non-null  object 
 10  plan_start_date         32950 non-null  object 
 11  plan_end_date           32950 non-null  object 
 12  budget_level3_name      840 non-null    object 
 13  budget_level3_key       840 non-null    object 
 14  budget_level2_key       32950 non-null

In [ ]:
main_data_set_all_fields.head(4)

,Unnamed: 0,member_key,updated_at,first_name,last_name,disabilities,SA1,price_zone_code,plan_key,plan_status,plan_start_date,plan_end_date,budget_level3_name,budget_level3_key,budget_level2_key,budget_level2_name,budget_level1_key,budget_level1_name,invoice_state,claim_state,allocated_amount,requested_amount,funded_amount,allocated_amount_total,requested_amount_total,funded_amount_total
0,0,00109970-7029-11eb-81d6-9d4df94b6224,2021-04,Wynny,Pressman,NaN,NaN,ACT_NSW_QLD_VIC,588427f1-e45d-4a6e-b307-9c68a13849ef,PLAN_DELIVERY_ACTIVE,2021-02,2024-02,NaN,NaN,7d0af53b-f597-11e9-bfb4-022d4762bb3c,Assistance with social and community participa...,58745466-f597-11e9-bfb4-022d4762bb3c,Core,ALL_PAID,PAID,50000.00,98.8000,98.80,272168.34,11316.7000,11316.70
1,1,00109970-7029-11eb-81d6-9d4df94b6224,2021-04,Wynny,Pressman,NaN,NaN,ACT_NSW_QLD_VIC,588427f1-e45d-4a6e-b307-9c68a13849ef,PLAN_DELIVERY_ACTIVE,2021-02,2024-02,NaN,NaN,7d0af71b-f597-11e9-bfb4-022d4762bb3c,Assistance with daily living,58745466-f597-11e9-bfb4-022d4762bb3c,Core,ALL_PAID,PAID,222168.34,11217.9000,11217.90,272168.34,11316.7000,11316.70
2,2,0010f240-3b5d-11eb-aa73-7be7bbd3c758,2021-04,Beryl,McDuff,Asperger syndrome,3138648.0,ACT_NSW_QLD_VIC,ef286fe8-8b36-4d77-97a0-59f9ef3df9b8,PLAN_DELIVERY_ACTIVE,2020-09,2021-09,NaN,NaN,7d0af57b-f597-11e9-bfb4-022d4762bb3c,Improved daily living skills,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,ALL_PAID,PAID,9801.60,2559.6384,2559.67,9801.60,2559.6384,2559.67
3,3,00117a50-cbc6-11ea-805a-75fc51d39a77,2021-04,Jethro,Perchard,NaN,NaN,ACT_NSW_QLD_VIC,8aa26238-d922-4963-bcaf-427d24a610bc,COMPLETED,2020-04,2021-04,NaN,NaN,7d0af57b-f597-11e9-bfb4-022d4762bb3c,Improved daily living skills,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,ALL_PAID,PAID,64524.00,8503.6500,8503.65,64524.00,8503.6500,8503.65


In [ ]:
main_data_No_NaN = main_data_set_all_fields.replace(np.nan, 'NA')

main_data_No_NaN = main_data_No_NaN[(main_data_No_NaN["plan_status"] == 'PLAN_DELIVERY_ACTIVE')]

In [ ]:
# Select just the fields to be used
main_data_set = main_data_No_NaN.groupby([
                                           "member_key"
                                         , "first_name"
                                         , "last_name"
                                         , "budget_level1_key"
                                         , "budget_level1_name"
                                         , "budget_level2_key"
                                         , "budget_level2_name"
                                         , "plan_start_date"
                                         , "plan_end_date"
                                         , "allocated_amount_total"
                                        ]).agg({"funded_amount": "sum", "allocated_amount": "sum"}).reset_index().copy()

In [ ]:
months_total = main_data_set[{
                              "member_key"
                            , "plan_start_date"
                            , "plan_end_date"}].copy()

months_total['plan_months_total'] = ((pd.to_datetime(months_total['plan_end_date']).dt.year
                                   -  pd.to_datetime(months_total['plan_start_date']).dt.year) * 12) + (pd.to_datetime(months_total['plan_end_date']).dt.month
                                   -  pd.to_datetime(months_total['plan_start_date']).dt.month)

months_total_merged = months_total[{"member_key", "plan_months_total"}]
                            
months_total_merged = months_total[{
                                    "member_key"
                                  , "plan_months_total"
                            }].groupby([
                                       "member_key"
                                     , "plan_months_total"
                                       ]).count().reset_index()

months_total_merged = pd.merge(main_data_set, months_total_merged, how="left", on=["member_key"])

In [ ]:
allocated_amount_month = months_total_merged[{
                                              "member_key"
                                            , "budget_level1_key"
                                            , "budget_level2_key"
                                            , "plan_months_total"
                                            , "allocated_amount"
                                            }].copy()

allocated_amount_month["allocated_amount_month"] = allocated_amount_month["allocated_amount"] / allocated_amount_month["plan_months_total"]                                                    

allocated_amount_month = allocated_amount_month[{'member_key', 'budget_level1_key', 'budget_level2_key', 'allocated_amount_month'}]

months_total_merged_monthly = pd.merge(months_total_merged, allocated_amount_month, how="left", on=["member_key", "budget_level1_key", "budget_level2_key"])

In [ ]:
month_today = main_data_set[{
                           "member_key"
                         , "budget_level1_key"
                         , "budget_level2_key"
                         , "plan_start_date"
                            }].copy()


month_today['month_actual'] = ((pd.to_datetime("today").year -  pd.to_datetime(month_today['plan_start_date']).dt.year) * 12)                                                                                                                     + (pd.to_datetime("today").month - pd.to_datetime(month_today['plan_start_date']).dt.month)

month_today = month_today[{'member_key', 'budget_level1_key', 'budget_level2_key', 'month_actual'}]

month_actual = pd.merge(months_total_merged_monthly, month_today, how="left", on=["member_key", "budget_level1_key", "budget_level2_key"])

month_actual["spent_amount_predicted"] = month_actual["allocated_amount_month"] * month_actual["month_actual"]

In [ ]:
month_actual['funded_amount'] = month_actual['funded_amount'].round(2)
month_actual['allocated_amount'] = month_actual['allocated_amount'].round(2)

month_actual['allocated_amount_month'] = month_actual['allocated_amount_month'].round(2)
month_actual['spent_amount_predicted'] = month_actual['spent_amount_predicted'].round(2)

month_actual['months_left'] = month_actual['plan_months_total'] - month_actual['month_actual']

In [ ]:
month_actual['funded_amount_percentage'] = (month_actual['funded_amount'] / month_actual['allocated_amount']) * 100#).round(2)

month_actual['months_left_percentage'] = (month_actual['months_left'] / month_actual['plan_months_total']) * 100#).round(2)

month_actual['funded_amount_months_left'] = month_actual['funded_amount_percentage'] / month_actual['months_left_percentage']#).round(2)

month_actual = month_actual.loc[
                                (month_actual["plan_months_total"] >= 12)
                              & (month_actual["allocated_amount"] > 0)
                              & (month_actual["months_left"] > 0)
                               ]

In [ ]:
main_data_set = month_actual[{
                              "member_key"
                            , "first_name" 
                            , "last_name" 
                            , "budget_level1_key"
                            , "budget_level1_name"
                            , "budget_level2_key"
                            , "budget_level2_name"
                            , "plan_months_total"
                            , "allocated_amount"
                            , "funded_amount_months_left"
                            }].copy()

In [ ]:
month_actual['funded_amount'] = month_actual['funded_amount'].round(2)
month_actual['allocated_amount'] = month_actual['allocated_amount'].round(2)

month_actual['allocated_amount_month'] = month_actual['allocated_amount_month'].round(2)
month_actual['spent_amount_predicted'] = month_actual['spent_amount_predicted'].round(2)

month_actual['months_left'] = month_actual['plan_months_total'] - month_actual['month_actual']

In [ ]:
 main_data_set['spending_status'] = np.where(main_data_set['funded_amount_months_left'] < 0.9, 'Overspending', 'Underspending')
 main_data_set['spending_status'] = np.where(((main_data_set['funded_amount_months_left'] >= 0.9) & (main_data_set['funded_amount_months_left'] <= 1.2)), 'On_Track', main_data_set['spending_status'])

In [ ]:
main_data_set = main_data_set[{
                              "member_key"
                            , "first_name" 
                            , "last_name" 
                            , "budget_level1_key"
                            , "budget_level1_name"
                            , "budget_level2_key"
                            , "budget_level2_name"
                            , "plan_months_total"
                            , "allocated_amount"
                            , "spending_status"
                            }].copy()

In [ ]:
df_model = main_data_set[{
                          'plan_months_total'
                        , 'allocated_amount'
                        , 'spending_status'
                        }].copy()

In [ ]:
# Total plan amount 
# Total plan lenght

X = df_model[{
                'plan_months_total'
              , 'allocated_amount'
            }].values

y = df_model['spending_status'].values

normalized_range = sklearn.preprocessing.MinMaxScaler(feature_range=(-1,1))
X = normalized_range.fit_transform(X)

In [ ]:
# TRAINING MODEL
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=10)

In [ ]:
#Feature scaling
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)

regression = LogisticRegression(random_state=0, multi_class='multinomial', penalty='none', solver='newton-cg', max_iter = 150, warm_start = False)

In [ ]:
skf = StratifiedKFold(n_splits=5)

In [ ]:
splitter = StratifiedShuffleSplit(n_splits=3, test_size=0.33, random_state=0)
lst_accu_stratified = []
   
for train_index, test_index in splitter.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = model.fit(X_train, y_train)
    lst_accu_stratified.append(model.score(X_test, y_test))
   
# Print the output.
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      mean(lst_accu_stratified)*100, '%')
print('\nStandard Deviation is:', stdev(lst_accu_stratified))

List of possible accuracy: [0.77652606759235, 0.77652606759235, 0.77652606759235]

Maximum Accuracy That can be obtained from this model is: 77.652606759235 %

Minimum Accuracy: 77.652606759235 %

Overall Accuracy: 77.652606759235 %

Standard Deviation is: 0.0


In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
unique1, counts1 = np.unique(y_test, return_counts=True)
print(dict(zip(unique, counts)))
print(dict(zip(unique1, counts1)))

{'On_Track': 429, 'Overspending': 6018, 'Underspending': 1301}
{'On_Track': 212, 'Overspending': 2964, 'Underspending': 641}


In [ ]:
model = model.fit(X_train, y_train)

In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
unique1, counts1 = np.unique(y_test, return_counts=True)
print(dict(zip(unique, counts)))
print(dict(zip(unique1, counts1)))

{'On_Track': 429, 'Overspending': 6018, 'Underspending': 1301}
{'On_Track': 212, 'Overspending': 2964, 'Underspending': 641}


In [ ]:
y_pred = model.predict(X_test)
confmtrx = np.array(confusion_matrix(y_test, y_pred))

pd.DataFrame(confmtrx, index=['Underspending','Overspending','On_Track'],
columns=['predicted_Underspending', 'predicted_Overspending', 'predicted_On_Track'])

,predicted_Underspending,predicted_Overspending,predicted_On_Track
Underspending,0,212,0
Overspending,0,2964,0
On_Track,0,641,0


In [ ]:
predictions = mlp.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

[[   0  212    0]
 [   0 2964    0]
 [   0  641    0]]
               precision    recall  f1-score   support

     On_Track       0.00      0.00      0.00       212
 Overspending       0.78      1.00      0.87      2964
Underspending       0.00      0.00      0.00       641

     accuracy                           0.78      3817
    macro avg       0.26      0.33      0.29      3817
 weighted avg       0.60      0.78      0.68      3817



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
